In [1]:
import pymongo
import pandas as pd
import numpy as np
from bson import ObjectId

import keyring
import mysql.connector

import arcgis
from arcgis.gis import GIS
import datetime
import getpass
from IPython.display import HTML

from arcgis.geocoding import geocode
from arcgis.features import Feature, FeatureSet
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [2]:
#pip install mysql-connector-python

In [3]:
password = keyring.get_password("arcgis", "Glodanale")
gis = GIS("https://www.arcgis.com", "Glodanale", password)

In [4]:
def queryChildren():
    # Connection details
    host = "localhost:27017"  # Replace with your MongoDB host
    port = 27017  # Default MongoDB port
    #username = "your-username"  # Replace with your username (if required)
    #password = "your-password"  # Replace with your password (if required)

    # Create a MongoDB client
    client = pymongo.MongoClient(host, port)

    # Choose a database
    db = client["EIDMS_Database"]  # Replace with your database name

    # Choose a collection within the database
    childRecordData = db["Child_Records"]  # Replace with your collection name

    matching_documents = childRecordData.find({})
    child_list = list(matching_documents)
    df = pd.DataFrame(child_list)
    return df, childRecordData

In [5]:
def queryUsers():
    # Connection details
    host = "localhost:27017"  # Replace with your MongoDB host
    port = 27017  # Default MongoDB port
    #username = "your-username"  # Replace with your username (if required)
    #password = "your-password"  # Replace with your password (if required)

    # Create a MongoDB client
    client = pymongo.MongoClient(host, port)

    # Choose a database
    db = client["EIDMS_Database"]  # Replace with your database

    # Choose a collection within the database
    userRecordData = db["Users"]

    matching_documents = userRecordData.find({})
    user_list = list(matching_documents)
    df = pd.DataFrame(user_list)
    print(df)
    return df, userRecordData

In [6]:
def childExtractData(df):

# Assuming df is your DataFrame
    address_data = []

    for index, row in df.iterrows():
        _id = row["_id"]
        
        name = row["Name"]
        FirstName = name["FirstName"]
        LastName = name["LastName"]
        
        freq = row["MeetingFrequency"]
        ServiceHours = freq["TimeEquivalent"]
        
        pg = row["PrimaryGuardian"]
        # Check if 'Address' key exists in the dictionary and is a dictionary
        if 'Address' in pg and isinstance(pg['Address'], dict):
            # Extract the individual address components
            Address1 = pg['Address'].get('Address1', '')
            City = pg['Address'].get('City', '')
            State = pg['Address'].get('State', '')
            Zipcode = pg['Address'].get('Zipcode', '')
            County = pg['Address'].get('County', '')
            AddressChanged = pg['Address'].get('AddressChanged', '')
        
            # Append the address components as a tuple to the list
            address_data.append((str(_id), FirstName, LastName, float(ServiceHours), Address1, City, State, Zipcode, County, AddressChanged))

# Create a new DataFrame from the address data
    address_df = pd.DataFrame(address_data, columns=['_id', 'FirstName', 'LastName', 'ServiceHours', 'Address1', 'City', 'State', 'Zipcode', 'County', 'AddressChanged'])
    address_df["Longitude"] = ''
    address_df["Latitude"] = ''

# address_df now contains the separated address attributes as columns
    print(address_df)
    return address_df

In [7]:
def userExtractData(df):
    address_data = []

    for index, row in df.iterrows():
        _id = row["_id"]
        
        name = row["Name"]
        FirstName = name["FirstName"]
        LastName = name["LastName"]
        
        TotalServiceHours = row["WorkLoad"]
        
        address = row["Address"]
        Address1 = address["Address1"]
        City = address["City"]
        State = address["State"]
        Zipcode = address["Zipcode"]
        County = address["County"]
        AddressChanged = address["AddressChanged"]
        
        # Append the address components as a tuple to the list
        address_data.append((str(_id), FirstName, LastName, float(TotalServiceHours), Address1, City, State, Zipcode, County, AddressChanged))
        
    # Create a new DataFrame from the address data
    address_df = pd.DataFrame(address_data, columns=['_id', 'FirstName', 'LastName', 'TotalServiceHours', 'Address1', 'City', 'State', 'Zipcode', 'County', 'AddressChanged'])
    address_df["Longitude"] = ''
    address_df["Latitude"] = ''

# address_df now contains the separated address attributes as columns
    print(address_df)
    return address_df

In [8]:
def coordinateAddresses(addresses):
    updatedDF = addresses.copy()
    updatedDF = updatedDF[updatedDF.AddressChanged == True]
    updatedDF = updatedDF.reset_index()
    length = updatedDF.shape[0]
    for i in range (0, length):
        if updatedDF.loc[i].isnull().any():
            print(f"{updatedDF.loc[i, 'FirstName']} {updatedDF.loc[i, 'LastName']}  contains null values")
            
        else:
            multi_field_address = {
                "Address" : updatedDF.loc[i, "Address1"],
                "City" : updatedDF.loc[i, "City"],
                "Region" : updatedDF.loc[i, "State"],
                "Postal" : updatedDF.loc[i, "Zipcode"]
            }
            geo = geocode(multi_field_address)
            print(f"GEO for {updatedDF.loc[i, 'FirstName']} {updatedDF.loc[i, 'LastName']}")
            #print(geo)
            geoData = geo[0]
            locationCoord = geoData["location"]
            longitude = locationCoord["x"]
            latitude = locationCoord["y"]
            updatedDF.at[i, "Longitude"] = longitude
            updatedDF.at[i, "Latitude"] = latitude            
            
    print(updatedDF)
    return updatedDF

In [9]:
def pushEIToMySQL(updatedDF, collection):
    if updatedDF.empty:
        print("No EI updates in dataset to push to the databases")
    else:
        connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
        )
    
        cursor = connection.cursor()
    
        print(f"\n\nDatabase connected\n\n")
    
        for index, row in updatedDF.iterrows():
            if row.isnull().any():
                print(f"{row['FirstName']} {row['LastName']} contains null values")
            else:
                print(f"Longitude: {row['Longitude']}")
                #mysql update
                insertQuery = ("INSERT INTO ei (ei_id, First_Name, Last_Name, Total_Service_Hours, Address, City, State, Zipcode, County, Longitude, Latitude) "
                               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) "
                               "ON DUPLICATE KEY UPDATE "
                               "First_Name = VALUES(First_Name), "
                               "Last_Name = VALUES(Last_Name), "
                               "Total_Service_Hours = VALUES(Total_Service_Hours), "
                               "Address = VALUES(Address), "
                               "City = VALUES(City), "
                               "State = VALUES(State), "
                               "Zipcode = VALUES(Zipcode), "
                               "County = VALUES(County), "
                               "Longitude = VALUES(Longitude), "
                               "Latitude = VALUES(Latitude);"
                              )
    
                cursor.execute(insertQuery, (
                    row['_id'], row["FirstName"], row["LastName"],
                    row["TotalServiceHours"], row["Address1"], row["City"],
                    row["State"], row["Zipcode"], row["County"],
                    row["Longitude"], row["Latitude"]
                ))
                print(f"{row['FirstName']} {row['LastName']} is inserted or updated in the MySQL database")
            
                #mongoDB update
                doc_id = ObjectId(updatedDF.loc[index, "_id"])
                filter = {"_id" : doc_id}
                
                #if collectionType == "User":
                update = {"$set": {"Address.AddressChanged": False}}
                #elif collectionType == "Child":
                    #update = {"$set": {"PrimaryGuardian.Address.AddressChanged": False}}
                    
                collection.update_one(filter, update)
                updatedDF.at[index, "AddressChanged"] = False
                print(f"{updatedDF.loc[index, 'FirstName']} {updatedDF.loc[index, 'LastName']} address status has been updated in MongoDB")
    
        connection.commit()
        cursor.close()
        connection.close()
    
        print("\n\n\n\n\n")
        print(updatedDF)

In [10]:
def pushChildToMySQL(updatedDF, collection):
    if updatedDF.empty:
        print("No child updates in dataset to push to the databases")
    else:
        connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
        )
    
        cursor = connection.cursor()
    
        print(f"\n\nDatabase connected\n\n")
    
        for index, row in updatedDF.iterrows():
            if row.isnull().any():
                print(f"{row['FirstName']} {row['LastName']} contains null values")
            else:
                print(f"Longitude: {row['Longitude']}")
                #mysql update
                insertQuery = ("INSERT INTO child (child_id, First_Name, Last_Name, Child_Service_Hours, Address, City, State, Zipcode, County, Longitude, Latitude) "
                               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) "
                               "ON DUPLICATE KEY UPDATE "
                               "First_Name = VALUES(First_Name), "
                               "Last_Name = VALUES(Last_Name), "
                               "Child_Service_Hours = VALUES(Child_Service_Hours), "
                               "Address = VALUES(Address), "
                               "City = VALUES(City), "
                               "State = VALUES(State), "
                               "Zipcode = VALUES(Zipcode), "
                               "County = VALUES(County), "
                               "Longitude = VALUES(Longitude), "
                               "Latitude = VALUES(Latitude);"
                              )
                
                try:
                    cursor.execute(insertQuery, (
                        row['_id'], row["FirstName"], row["LastName"],
                        row["ServiceHours"], row["Address1"], row["City"],
                        row["State"], row["Zipcode"], row["County"],
                        row["Longitude"], row["Latitude"]
                    ))
                    print(f"{row['FirstName']} {row['LastName']} is inserted or updated in the MySQL database")
            
                    #mongoDB update
                    doc_id = ObjectId(updatedDF.loc[index, "_id"])
                    filter = {"_id" : doc_id}

                    update = {"$set": {"PrimaryGuardian.Address.AddressChanged": False}}
                    
                    collection.update_one(filter, update)
                    updatedDF.at[index, "AddressChanged"] = False
                    print(f"{updatedDF.loc[index, 'FirstName']} {updatedDF.loc[index, 'LastName']} address status has been updated in MongoDB")
                
                except mysql.connector.Error as err:
                    print(f"Error: {err}")
    
        connection.commit()
        cursor.close()
        connection.close()
    
        print("\n\n\n\n\n")
        print(updatedDF)

In [11]:
def pullDataFromMySQL():
    connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
    )
    
    cursor = connection.cursor()
    
    query = "SELECT * FROM ei"
    cursor.execute(query)
    
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
        
    cursor.close()
    connection.close()

In [12]:
def setAllAddressCoordinates():
    dataC, childRecordData = queryChildren()
    print("Children have been queried------------------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")
    dataU, userRecordData = queryUsers()
    print("Users have been queried---------------------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")
    dataExC = childExtractData(dataC)
    print("Child data extracted------------------------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")
    dataExU = userExtractData(dataU)
    print("User data extracted-------------------------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")
    newChildData = coordinateAddresses(dataExC)
    print("Child addresses coordinated-----------------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")
    newUserData = coordinateAddresses(dataExU)
    print("User addresses coordinated------------------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")
    print(newChildData.info())
    pushChildToMySQL(newChildData, childRecordData)
    print("Child data pushed to MySQL and MongoDB------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")
    pushEIToMySQL(newUserData, userRecordData)
    print("User data pushed to MySQL and MongoDB-------------------------------------------------------------------------------------------------------------------------------")

In [13]:
dataC, childRecordData = queryChildren()
print("Children have been queried------------------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")
dataU, userRecordData = queryUsers()
print("Users have been queried---------------------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")

Children have been queried------------------------------------------------------------------------------------------------------------------------------------------





                         _id                  Email  Role  WorkLoad  \
0   65aa8ce6aa6f6de738b0f748      GLOVERAD@ETSU.EDU     0         0   
1   65d75a2ce09e67e025e8cd30  RichardsonMG@etsu.edu     2         0   
2   65d75bb0e09e67e025e8cd31       TiptonK@etsu.edu     0         1   
3   65d75c59e09e67e025e8cd32       KnightK@etsu.edu     1         0   
4   65d75c87e09e67e025e8cd33       TaylorR@etsu.edu     1         0   
5   65d75cb9e09e67e025e8cd34      VanoverB@etsu.edu     2         0   
6   65d75ce1e09e67e025e8cd35      AilshieB@etsu.edu     1         0   
7   65d75d08e09e67e025e8cd36      MartinoD@etsu.edu     2         0   
8   65d75d4ce09e67e025e8cd37       CrumlyC@etsu.edu     2         0   
9   65d75d80e09e67e025e8cd38       RisnerK@etsu.edu     2         0   
10  65d75da9e09e67e025e8cd39         UgwoL@etsu.e

In [14]:
dataExC = childExtractData(dataC)
print("Child data extracted------------------------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")
dataExU = userExtractData(dataU)
print("User data extracted-------------------------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")

                          _id FirstName    LastName  ServiceHours  \
0    65d76430e09e67e025e8cd43      John      Fauver           1.0   
1    65d764dde09e67e025e8cd44     River     Hefflin           0.5   
2    65d766f6e09e67e025e8cd45       Kai  Khaysavang           0.5   
3    65d76799e09e67e025e8cd46    Robert      Richie           1.0   
4    65d76829e09e67e025e8cd47   Matilda     Withrow           0.5   
..                        ...       ...         ...           ...   
191  65e6038938f826f29267202f     Judah     Overbay           1.0   
192  65e603f238f826f292672030    Bianca       Pogue           1.0   
193  65e6044938f826f292672031   Emorett     Woodall           1.0   
194  65e6049b38f826f292672032      Remi       Woody           0.5   
195  65ef076606e0a866af56d202  Madilynn      Phipps           1.0   

                      Address1           City State Zipcode    County  \
0              237 Peters Road        Bristol    TN   37620  Sullivan   
1          613 Meadowview

In [15]:
newChildData = coordinateAddresses(dataExC)
print("Child addresses coordinated-----------------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")
newUserData = coordinateAddresses(dataExU)
print("User addresses coordinated------------------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")

Empty DataFrame
Columns: [index, _id, FirstName, LastName, ServiceHours, Address1, City, State, Zipcode, County, AddressChanged, Longitude, Latitude]
Index: []
Child addresses coordinated-----------------------------------------------------------------------------------------------------------------------------------------





GEO for Lydia Ugwu
   index                       _id FirstName LastName  TotalServiceHours  \
0     10  65d75da9e09e67e025e8cd39     Lydia     Ugwu                0.0   

                  Address1          City State Zipcode      County  \
0  801 Magnolia Avenue Ext  Johnson City    TN   37604  Washington   

   AddressChanged  Longitude   Latitude  
0            True -82.354896  36.299501  
User addresses coordinated------------------------------------------------------------------------------------------------------------------------------------------







In [16]:
print(newChildData.info())
pushChildToMySQL(newChildData, childRecordData)
print("Child data pushed to MySQL and MongoDB------------------------------------------------------------------------------------------------------------------------------\n\n\n\n\n")
pushEIToMySQL(newUserData, userRecordData)
print("User data pushed to MySQL and MongoDB-------------------------------------------------------------------------------------------------------------------------------")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           0 non-null      int64  
 1   _id             0 non-null      object 
 2   FirstName       0 non-null      object 
 3   LastName        0 non-null      object 
 4   ServiceHours    0 non-null      float64
 5   Address1        0 non-null      object 
 6   City            0 non-null      object 
 7   State           0 non-null      object 
 8   Zipcode         0 non-null      object 
 9   County          0 non-null      object 
 10  AddressChanged  0 non-null      bool   
 11  Longitude       0 non-null      object 
 12  Latitude        0 non-null      object 
dtypes: bool(1), float64(1), int64(1), object(10)
memory usage: 124.0+ bytes
None
No child updates in dataset to push to the databases
Child data pushed to MySQL and MongoDB-----------------------------------------------------------

In [17]:
pullDataFromMySQL()

('65d75a2ce09e67e025e8cd30', 'Mary', 'Richardson', 0.0, '904 West Hillcrest Drive', 'Johnson City', 'TN', '37604', 'Washington', -82.37210845947266, 36.31948471069336)
('65d75bb0e09e67e025e8cd31', 'Kirstin', 'Tipton', 1.0, '151 Valley View Drive', 'Abingdon', 'VA', '24210', 'Washington', -81.977835996997, 36.722124478185)
('65d75c59e09e67e025e8cd32', 'Kassi', 'Knight', 0.0, '7040 Blue Springs Parkway', 'Mosheim', 'TN', '37818', 'Greene', -82.94459533691406, 36.197052001953125)
('65d75c87e09e67e025e8cd33', 'Rachel', 'Taylor', 0.0, '502 Nunley Drive', 'Johnson City', 'TN', '37604', 'Washington', -82.4153060913086, 36.30536651611328)
('65d75cb9e09e67e025e8cd34', 'Brannan', 'Vanover', 0.0, '1573 Old Bristol Highway', 'Elizabethton', 'TN', '37643', 'Carter', -82.2342300415039, 36.391361236572266)
('65d75ce1e09e67e025e8cd35', 'Briana', 'Ailshie', 0.0, '1370 Riverbend Drive', 'Kingsport', 'TN', '37664', 'Sullivan', -82.5220947265625, 36.50907897949219)
('65d75d08e09e67e025e8cd36', 'Dawn', 'Ma